In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime
import time
from dotenv import load_dotenv
import os
print('All necessary libraries as installed!')

All necessary libraries as installed!


In [2]:
# Configure visualizations

%matplotlib inline
sns.set(style="whitegrid")
print('viz configured!')

viz configured!


In [3]:
# Load .env file
load_dotenv()

# Access Square API Token
ACCESS_TOKEN = os.getenv("SQUARE_ACCESS_TOKEN")
if not ACCESS_TOKEN:
    raise ValueError("Square API TOKEN not found in .env file!")

PAYMENTS_API_URL = 'https://connect.squareup.com/v2/payments'
ORDERS_API_URL = 'https://connect.squareup.com/v2/orders'
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Content-Type": "application/json"
}





In [4]:
def fetch_payment_data(start_date, end_date):
    response = requests.get(
        PAYMENTS_API_URL,
        headers=headers,
        params={
            "begin_time": f"{start_date}T00:00:00Z",
            "end_time": f"{end_date}T23:59:59Z"
        }
    )
    if response.status_code == 200:
        return pd.json_normalize(response.json().get("payments", []))
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return pd.DataFrame()
    
start_date = "2024-01-01"
end_date = "2024-01-30"
payments_data = fetch_payment_data(start_date, end_date)




In [5]:
valid_payments = payments_data.dropna(subset=["order_id"])
def fetch_order_details(order_id):
    try:
        response = requests.get(f"{ORDERS_API_URL}/{order_id}", headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching order {order_id}: {response.status_code} - {response.text}")
            return {}
    except Exception as e:
        print(f"Exception occured for order {order_id}: {e}")
        return {}

In [6]:

order_details = []
for order_id in payments_data['order_id']:
    order = fetch_order_details(order_id)
    if order:
        for item in order.get('line_items', []):
            order_details.append({
                "order_id": order_id,
                "item_name": item["name"],
                "quantity": item["quantity"],
                "gross_sales": item["gross_sales_money"]["amount"] / 100, 
                "category": item.get("category_name", "Unknown")
            })
    
    time.sleep(0.3)







In [7]:
order_df = pd.DataFrame(order_details)

if 'order_id' in payments_data.columns and 'order_id' in order_df.columns:
    enriched_data = pd.merge(payments_data, order_df, on="order_id", how="left")
else: 
    print("The 'order_id' column is missiong in one or more Dataframes.")
    enriched_data = payments_data

enriched_data.to_csv("enriched_data.csv", index=False)

print("Enriched data has been saved to 'enriched_data.csv'")

enriched_data_preview = pd.read_csv("enriched_data.csv")
enriched_data_preview.head()

The 'order_id' column is missiong in one or more Dataframes.
Enriched data has been saved to 'enriched_data.csv'


,id,created_at,updated_at,status,source_type,location_id,order_id,processing_fee,customer_id,employee_id,...,buyer_email_address,delay_action,delayed_until,risk_evaluation.created_at,risk_evaluation.risk_level,shipping_address.first_name,shipping_address.last_name,application_details.application_id,billing_address.postal_code,billing_address.country
0,Ja9BICk5UrkN1NtDDaTtKEt6vaB,2024-01-30T20:52:21.389Z,2024-01-30T20:52:29.932Z,COMPLETED,CARD,LW76KW1EJ9K7N,llWLPuv4Qf4zeHEh2XmSmr0eV,"[{'effective_at': '2024-01-30T22:52:29.000Z', ...",ZCY835P01S5WDBD793RPR55H7R,TMSV3D5FpfPCwBUa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,lioAcYpgUBpVAXT3vv05cEUEuaB,2024-01-30T20:41:59.495Z,2024-01-30T20:42:01.100Z,COMPLETED,CASH,LW76KW1EJ9K7N,1h8559Rphx3c3GhsWLQtJcfeV,NaN,NaN,TMSV3D5FpfPCwBUa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LF61s4rgmRCuPMwGOQAdGR5FvaB,2024-01-30T20:27:34.039Z,2024-01-30T20:27:35.551Z,COMPLETED,CASH,LW76KW1EJ9K7N,HARIObYtxyvw80puKOfrjK9eV,NaN,NaN,TMSV3D5FpfPCwBUa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bV8Y8P5HS8HPDl21b138PHpcuaB,2024-01-30T20:16:21.575Z,2024-01-30T20:16:30.869Z,COMPLETED,CARD,LW76KW1EJ9K7N,RVtymxI9W9vYOAC331gNykteV,"[{'effective_at': '2024-01-30T22:16:27.000Z', ...",T3MCC6W38S5H542409G4SC5A94,TMSV3D5FpfPCwBUa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,VoPyRkACtYEvOE8syW5ZXpBSuaB,2024-01-30T20:13:27.845Z,2024-01-30T20:13:36.560Z,COMPLETED,CARD,LW76KW1EJ9K7N,jOVCuhWg0GvTATmlRpbi7FgeV,"[{'effective_at': '2024-01-30T22:13:35.000Z', ...",42H7MZZBSMJFFXB2K8PKSAS2QC,TMSV3D5FpfPCwBUa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
enriched_data_preview.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 69 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   id                                                    100 non-null    object 
 1   created_at                                            100 non-null    object 
 2   updated_at                                            100 non-null    object 
 3   status                                                100 non-null    object 
 4   source_type                                           100 non-null    object 
 5   location_id                                           100 non-null    object 
 6   order_id                                              100 non-null    object 
 7   processing_fee                                        71 non-null     object 
 8   customer_id                                           74 non-